In [ ]:
import os, sys
path_to_scr_folder=os.path.join(os.path.dirname(os.path.abspath('')), 'src')
sys.path.append(path_to_scr_folder)

In [ ]:
import read_chroma
from peak_detection import peak_detection
import numpy as np
import projection
import plot
import baseline_correction
import netCDF4 as nc
import matplotlib.pyplot as plt
import peak_detection
from kneed import KneeLocator
import skimage

In [ ]:
import sys
!{sys.executable} -m pip install kneed

In [ ]:

file_path = '/home/camille/Documents/app/data/input/A-F-028-817822-droite-ReCIVA.cdf'

In [ ]:
    
# get_mod_time(file_path):
data = nc.Dataset(file_path, 'r')
scan_number = data.dimensions['scan_number'].size
if scan_number == 328125:   
    mod_time = 1.25
    print("type de donnees: G0/plasma")
elif scan_number == 540035:
    mod_time = 1.7
    print("type de donnnees: air expire")
else:
    print("scan_number non reconnu")
mod_time
# return mod_time

In [ ]:
tic_chromato, time_rn, spectra_obj = read_chroma.read_chroma(file_path, mod_time, None)

In [ ]:

plot.visualizer((tic_chromato, time_rn), title="chromato_tic", log_chromato=False,)

In [ ]:
# evaluate best filtering factor on chromato tic after preprocessing +  peak detection with peak local max
def evaluate_filtering_factors(chromato, sigma, filtering_factors,):
    results = []

    for ff in filtering_factors:
        threshold = ff * sigma * 100 / np.max(chromato)
            # if chromatogram is very noisy : avoid detecting noise as if it were real
            # peaks.
            # if chonmatogram  is very clean: detect weaker peaks
        peaks = peak_detection.peak_detection((chromato, time_rn, None), spectra=None, chromato_cube=None, dynamic_threshold_fact=threshold, ABS_THRESHOLDS=None, method="peak_local_max", mode="tic", cluster=False, unique=False)
        # peaks = peak_detection.peak_detection((chromato, time_rn, None), spectra=None, chromato_cube=None, dynamic_threshold_fact=threshold, ABS_THRESHOLDS=None, method="persistent_homology", mode="tic", cluster=False, unique=False)
        results.append(len(peaks))

    return results

filtering_factors = np.linspace(0.1, 1.0, 20)  # 20 valeurs entre 0.1 et 1.0

chromato_tic_preprocessed = baseline_correction.chromato_reduced_noise(tic_chromato)
sigma = skimage.restoration.estimate_sigma(chromato_tic_preprocessed, channel_axis=None)

n_peaks = evaluate_filtering_factors(chromato_tic_preprocessed, sigma, filtering_factors)


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(filtering_factors, n_peaks, marker='o')
plt.title("Nombre de pics détectés vs Filtering Factor")
plt.xlabel("Filtering Factor")
plt.ylabel("Nombre de pics détectés")
plt.grid(True)
plt.show()

In [ ]:
knee = KneeLocator(filtering_factors, n_peaks, curve='convex', direction='decreasing')
print(f"Filtering factor optimal ≈ {knee.knee}")